In [1]:
!/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip
!pip install tensorflow
!pip install keras

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages (20.1.1)


In [2]:
import numpy as np

import os
import cv2
import glob

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model, save_model
from keras.layers import Input, GlobalAveragePooling2D, GlobalMaxPooling2D
import keras.backend as K
from keras.models import Model, load_model
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical

In [3]:
train_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'train')
index_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'index')
test_dir = os.path.join('/GoogleLandmarkRetrieval2020', 'test')

In [4]:
print(train_dir)

/GoogleLandmarkRetrieval2020/train


In [5]:
batch_size = 20
epochs = 1 
IMG_HEIGHT = 28
IMG_WIDTH = 28

In [6]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [7]:
train = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory='train/0/0',
                                                           shuffle=False,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

Found 6120 images belonging to 16 classes.


In [8]:
val = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory='test',
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

Found 1129 images belonging to 16 classes.


In [9]:
X_train, _ = next(train)
X_test, _ = next(val)
X_train = X_train.reshape(60, 784)
X_test = X_test.reshape(60, 784)
print(X_train)

[[0.44705886 0.5411765  0.6901961  ... 0.62352943 0.4156863  0.75294125]
 [0.63529414 0.34901962 0.78823537 ... 0.6666667  0.48627454 0.5176471 ]
 [0.5921569  0.60784316 0.6392157  ... 0.10980393 0.10588236 0.09019608]
 ...
 [0.2392157  0.24313727 0.07058824 ... 0.29411766 0.17254902 0.9176471 ]
 [0.7960785  0.5803922  0.49803925 ... 0.65882355 0.3019608  0.2901961 ]
 [0.23137257 0.40000004 0.33333334 ... 0.00392157 0.01176471 0.        ]]


In [12]:
# Flattening the images from the 28x28 pixels to 1D 787 pixels

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
#X_train /= 255
#X_test /= 255


# building a linear stack of layers with the sequential model
model = Sequential()
# hidden layer
model.add(Dense(100, input_shape=(784,), activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

# looking at the model summary
model.summary()
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# training the model for 10 epochs
model.fit(X_train, batch_size=20, epochs=1, validation_data=(X_test))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


ValueError: in user code:

    /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:541 train_step  **
       